# 2바이트 한글 번역기


In [2]:
import os
# get txt files in current directory
files = [f for f in os.listdir('.') if f.endswith('.txt')]
files

['han.txt', 'honux.txt', 'mcom.txt', 'test2.txt']

In [5]:
ba = []
for file in files:
    with open(file, 'rb') as f:
        ba.append(f.read())


## 2바이트 조합형 한글 읽기

In [17]:
def divideJaso(b1, b2):
    """
    한글 음절을 초성, 중성, 종성으로 분리

    :param b1: 한글 음절의 첫번째 바이트
    :param b2: 한글 음절의 두번째 바이트
    """
    if b1 & 0x80 == 0:
        raise Exception("b1, b2 is not Korean")
    
    b51 = (b1 >> 2) & 0x1F  # 처음 5비트
    b52 = (b1 & 0x03) << 3 | b2 >> 5 # 중간 5비트 (2 + 3비트)
    b53 = b2 & 0x1F # 마지막 5비트    
    return (b51, b52, b53)

## 초성중성조합 합치기


In [12]:
CHOSUNG = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
JUNGSUNG = [0, 1, 2, 3, 4, 5, 6, 5, 6, 9, 8, 9, 12, 13, 12, 15, 16, 13, 14, 19, 20, 20, 20, 23, 18, 25, 20, 20]
JONGSUNG = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 17, 18, 20, 20, 21, 22, 24, 25, 26, 27]    

In [25]:
def composeHangul(b51, b52, b53):
        i1 = CHOSUNG[b51- 2]
        i2 = JUNGSUNG[b52 - 3]
        i3 = JONGSUNG[b53 - 1]        
        unicode = 0xAC00 + (i1 * 21 + i2) * 28 + i3
        # print(b51, b52, b53, i1, i2, i3, chr(unicode))    
        return chr(unicode)

In [26]:

def readHan(ba):
    res = []
    i = 0
    while i < len(ba):
        # 첫 비트가 1이면 2바이트를 읽어 한글로 변환
        if ba[i] & 0x80:
            cho, jung, jong = divideJaso(ba[i], ba[i+1])
            han = composeHangul(cho, jung, jong) 
            res.append(han)
            i += 2
        else:
            res.append(chr(ba[i]))
            i += 1
    return "".join(res)

In [28]:
print(readHan(ba[0]))

1234567890 aaa bbb ccc 가가가 가나다 하하하 aaa
123 aaa bbb ccc 가나다 가갸거겨 각간갇 아안앋악 aaa




In [29]:
for b in ba:
    print(readHan(b))
    print("===\n")

1234567890 aaa bbb ccc 가가가 가나다 하하하 aaa
123 aaa bbb ccc 가나다 가갸거겨 각간갇 아안앋악 aaa


===

안녕하세위?
문서입니다.
abcdefghijklmn
1 2 3 4 5 6 7 8 9 0


===

마법 이야기
마법을 배우기 이해
졸업시험을 보는 '마이컴'
졸업시험저 내윙은 마법쟈치원
한 가운데에 이치한
탑에 들어가 3개저 마법구슬을 가져오는 겄이다.
'마이컴'은 과연
3개저 마법구슬을 가져올 수 있는지

===

                                     애  국   가
                                                                         안익태
  동해물과 백두산이 마르고 닳도록
  하느님이 보우하사 우리나라 만세

  무궁화 삼천리 화려 강산
  대한사람 대한으로 길이 보전하세

  타잔이 10원짜리 팬티를 입지 않았다.

  Amazing grace, how sweet the sound
  That saves a wretch; like me!
  I one was lost, but now am found,
  Was blind, but now I see.
  \123,456,789 !@#$%^&*()_+


===

